### Data Ingestion

In [5]:
### Document Structure

from langchain_core.documents import Document

In [6]:
doc = Document(
    page_content = "this is the main text content I am using to create RAG",
    metadata = {
        "source":"example.txt",
        "pages": 1,
        "author": "Pavani",
        "data_created": "2025-11-25"
    }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'Pavani', 'data_created': '2025-11-25'}, page_content='this is the main text content I am using to create RAG')

In [7]:
### create a simple txt file
import os
os.makedirs("../data/text_files",exist_ok=True)

In [8]:
sample_texts = {
    "../data/text_files/python_intro.txt":"""Python Programming Introduction
    
    Python is a high-level, general-purpose programming language known for its clean, human-readable syntax. 
    It was created by Guido van Rossum and first released in 1991. 
    Today, it’s one of the most widely used languages in the world—popular among beginners and professionals alike.
    
    Why Python Is Popular

    Easy to read and write: Its syntax is close to English, making it beginner-friendly.

    Versatile: Used in web development, data science, machine learning, automation, AI, scientific computing, cybersecurity, and more.

    Large ecosystem: Thousands of libraries like NumPy, Pandas, TensorFlow, Django, Flask, and PyTorch.

    Cross-platform: Works on Windows, macOS, Linux, and more.

    Strong community: Millions of developers and tons of learning resources.
    """
}

for filepath, content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("sample text files created!")

sample text files created!


In [9]:
### TextLoader
# from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader=TextLoader("../data/text_files/python_intro.txt", encoding="utf-8")
document=loader.load()
print(document)


[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\n    Python is a high-level, general-purpose programming language known for its clean, human-readable syntax. \n    It was created by Guido van Rossum and first released in 1991. \n    Today, it’s one of the most widely used languages in the world—popular among beginners and professionals alike.\n\n    Why Python Is Popular\n\n    Easy to read and write: Its syntax is close to English, making it beginner-friendly.\n\n    Versatile: Used in web development, data science, machine learning, automation, AI, scientific computing, cybersecurity, and more.\n\n    Large ecosystem: Thousands of libraries like NumPy, Pandas, TensorFlow, Django, Flask, and PyTorch.\n\n    Cross-platform: Works on Windows, macOS, Linux, and more.\n\n    Strong community: Millions of developers and tons of learning resources.\n    ')]


In [11]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt", ## Pattern to match files
    loader_cls=TextLoader, ##loader class to use
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=False
)
documents=dir_loader.load()
documents

[Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\n    Python is a high-level, general-purpose programming language known for its clean, human-readable syntax. \n    It was created by Guido van Rossum and first released in 1991. \n    Today, it’s one of the most widely used languages in the world—popular among beginners and professionals alike.\n\n    Why Python Is Popular\n\n    Easy to read and write: Its syntax is close to English, making it beginner-friendly.\n\n    Versatile: Used in web development, data science, machine learning, automation, AI, scientific computing, cybersecurity, and more.\n\n    Large ecosystem: Thousands of libraries like NumPy, Pandas, TensorFlow, Django, Flask, and PyTorch.\n\n    Cross-platform: Works on Windows, macOS, Linux, and more.\n\n    Strong community: Millions of developers and tons of learning resources.\n    ')]

In [12]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/pdf",
    glob="**/*.pdf", ## Pattern to match files
    loader_cls=PyMuPDFLoader, ##loader class to use
    show_progress=False
)
pdf_documents=dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2021-06-25T20:13:04+05:30', 'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'total_pages': 14, 'format': 'PDF 1.7', 'title': 'FORWARD PROPAGATION', 'author': 'jahnavi Narindi', 'subject': '', 'keywords': '', 'moddate': '2021-06-25T20:13:04+05:30', 'trapped': '', 'modDate': "D:20210625201304+05'30'", 'creationDate': "D:20210625201304+05'30'", 'page': 0}, page_content='FORWARD \nPROPAGATION'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2021-06-25T20:13:04+05:30', 'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'total_pages': 14, 'format': 'PDF 1.7', 'title': 'FORWARD PROPAGATION', 'author': 'jahnavi Narindi'

In [13]:
type(pdf_documents[0])

langchain_core.documents.base.Document

In [17]:
### Text splitting get into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [18]:
chunks=split_documents(pdf_documents)
chunks

Split 175 documents into 93 chunks

Example chunk:
Content: FORWARD 
PROPAGATION...
Metadata: {'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2021-06-25T20:13:04+05:30', 'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'total_pages': 14, 'format': 'PDF 1.7', 'title': 'FORWARD PROPAGATION', 'author': 'jahnavi Narindi', 'subject': '', 'keywords': '', 'moddate': '2021-06-25T20:13:04+05:30', 'trapped': '', 'modDate': "D:20210625201304+05'30'", 'creationDate': "D:20210625201304+05'30'", 'page': 0}


[Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2021-06-25T20:13:04+05:30', 'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'total_pages': 14, 'format': 'PDF 1.7', 'title': 'FORWARD PROPAGATION', 'author': 'jahnavi Narindi', 'subject': '', 'keywords': '', 'moddate': '2021-06-25T20:13:04+05:30', 'trapped': '', 'modDate': "D:20210625201304+05'30'", 'creationDate': "D:20210625201304+05'30'", 'page': 0}, page_content='FORWARD \nPROPAGATION'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2021-06-25T20:13:04+05:30', 'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'total_pages': 14, 'format': 'PDF 1.7', 'title': 'FORWARD PROPAGATION', 'author': 'jahnavi Narindi'

### Embedding and VectorStoreDB

In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generation embeddings for a list o texts
        
        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generating embeddings with shape: {embeddings.shape}")
        return embeddings

## initialize the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [15]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the chromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize chromaDB client and collection"""
        try:
            #create persistent chromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            #Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collections: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """ 
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")

        #prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            #Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore


Vector store initialized. Collections: pdf_documents
Existing documents in collection: 0


In [19]:
chunks

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2021-06-25T20:13:04+05:30', 'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'total_pages': 14, 'format': 'PDF 1.7', 'title': 'FORWARD PROPAGATION', 'author': 'jahnavi Narindi', 'subject': '', 'keywords': '', 'moddate': '2021-06-25T20:13:04+05:30', 'trapped': '', 'modDate': "D:20210625201304+05'30'", 'creationDate': "D:20210625201304+05'30'", 'page': 0}, page_content='FORWARD \nPROPAGATION'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2021-06-25T20:13:04+05:30', 'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf', 'total_pages': 14, 'format': 'PDF 1.7', 'title': 'FORWARD PROPAGATION', 'author': 'jahnavi Narindi'

In [20]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 93 texts...


Batches: 100%|██████████| 3/3 [00:01<00:00,  1.68it/s]


Generating embeddings with shape: (93, 384)
Adding 93 documents to vector store...
Successfully added 93 documents to vector store
Total documents in collection: 93


### Retriever Pipeline From VectorStore

In [21]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [22]:
rag_retriever

In [23]:
rag_retriever.retrieve("What is forward propagation")

Retrieving documents for query: 'What is forward propagation'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 42.08it/s]

Generating embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_230836af_0',
  'content': 'FORWARD \nPROPAGATION',
  'metadata': {'moddate': '2021-06-25T20:13:04+05:30',
   'subject': '',
   'keywords': '',
   'trapped': '',
   'producer': 'Microsoft® PowerPoint® 2016',
   'creator': 'Microsoft® PowerPoint® 2016',
   'doc_index': 0,
   'total_pages': 14,
   'format': 'PDF 1.7',
   'creationDate': "D:20210625201304+05'30'",
   'title': 'FORWARD PROPAGATION',
   'content_length': 20,
   'author': 'jahnavi Narindi',
   'modDate': "D:20210625201304+05'30'",
   'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf',
   'creationdate': '2021-06-25T20:13:04+05:30',
   'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf',
   'page': 0},
  'similarity_score': 0.7536109238862991,
  'distance': 0.24638907611370087,
  'rank': 1},
 {'id': 'doc_20877837_11',
  'content': 'Forward propagation (or forward pass) refers to the calculation and storage of \nintermediate variables (including outputs) for a neural network in o

In [24]:
rag_retriever.retrieve("What is power BI")

Retrieving documents for query: 'What is power BI'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 53.87it/s]

Generating embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_09589929_13',
  'content': 'Power-BI\nPower-BI\nM a s t e r\ni n  1 5 - D a y s',
  'metadata': {'source': '..\\data\\pdf\\Master Power-BI in 15 days.pdf',
   'subject': '',
   'moddate': '',
   'content_length': 48,
   'page': 0,
   'title': '',
   'keywords': '',
   'creationdate': '',
   'author': '',
   'trapped': '',
   'producer': '',
   'file_path': '..\\data\\pdf\\Master Power-BI in 15 days.pdf',
   'creator': '',
   'format': 'PDF 1.5',
   'total_pages': 12,
   'modDate': '',
   'doc_index': 13,
   'creationDate': ''},
  'similarity_score': 0.33789098262786865,
  'distance': 0.6621090173721313,
  'rank': 1},
 {'id': 'doc_a404e096_16',
  'content': 'Stage 1:\nDay 1\nIntroduction to Power BI\nDay 1\nIntroduction to Power BI\nDay 2\nPractice Questions\nDay 2\nPractice Questions\n1',
  'metadata': {'creationDate': '',
   'subject': '',
   'file_path': '..\\data\\pdf\\Master Power-BI in 15 days.pdf',
   'modDate': '',
   'format': 'PDF 1.5',
   'moddate': '',
   'title

### RAG Pipeline- VectorDB To LLM Output Generation

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv("GROQ_API_KEY"))

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


In [33]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

In [34]:
class GroqLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)

        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    

In [35]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: gemma2-9b-it
Groq LLM initialized successfully!


In [36]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("forward propagation")

Retrieving documents for query: 'forward propagation'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 54.78it/s]

Generating embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_230836af_0',
  'content': 'FORWARD \nPROPAGATION',
  'metadata': {'modDate': "D:20210625201304+05'30'",
   'moddate': '2021-06-25T20:13:04+05:30',
   'content_length': 20,
   'title': 'FORWARD PROPAGATION',
   'producer': 'Microsoft® PowerPoint® 2016',
   'creator': 'Microsoft® PowerPoint® 2016',
   'keywords': '',
   'format': 'PDF 1.7',
   'subject': '',
   'creationdate': '2021-06-25T20:13:04+05:30',
   'author': 'jahnavi Narindi',
   'total_pages': 14,
   'source': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf',
   'file_path': '..\\data\\pdf\\FORWARDPROPAGATION_pdf-210709-181239.pdf',
   'page': 0,
   'creationDate': "D:20210625201304+05'30'",
   'trapped': '',
   'doc_index': 0},
  'similarity_score': 0.9999999999997548,
  'distance': 2.452111864253981e-13,
  'rank': 1},
 {'id': 'doc_20877837_11',
  'content': 'Forward propagation (or forward pass) refers to the calculation and storage of \nintermediate variables (including outputs) for a neural network in

### Integration Vectordb Context pipeline With LLM output

In [45]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    try:
        response = llm.invoke([prompt])  # or llm.invoke([prompt.format(...)]) if you prefer
        return getattr(response, "content", response)  # defensive: response.content if available
    except Exception as e:
        # print full exception so you can debug (BadRequestError will include server JSON)
        print("Error type:", type(e))
        try:
            # some clients include a `.response` with JSON body
            print("Response detail:", e.response.json())
        except Exception:
            print("Exception (no response JSON):", e)
        raise

In [46]:
answer=rag_simple("What is forward propagation?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is forward propagation?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 78.47it/s]

Generating embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


Forward propagation refers to the calculation and storage of intermediate variables for a neural network, from the input layer to the output layer.


### Enhanced RAG Pipeline Features

In [48]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Hard Negative Mining Technqiues", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'Hard Negative Mining Technqiues'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 38.90it/s]

Generating embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
Answer: No relevant context found.
Sources: []
Confidence: 0.0
Context Preview: 


In [49]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is attention is all you need", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what is attention is all you need'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 34.71it/s]

Generating embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)



Final Answer: No relevant context found.
Summary: There is no information to summarize as the provided text does not contain any relevant context.
History: {'question': 'what is attention is all you need', 'answer': 'No relevant context found.', 'sources': [], 'summary': 'There is no information to summarize as the provided text does not contain any relevant context.'}
